In [ ]:
#For VSCODE
#!pip install ipykernel
#!pip install ipywidgets

!pip install git+https://github.com/openai/swarm.git
# !pip install openai==1.55.3
# !pip install httpx==0.27.2
!pip install --force-reinstall -v "openai==1.55.3"
!pip install python-dotenv

In [ ]:
import os
# import httpx
from swarm import Swarm, Agent
from openai import OpenAI
from IPython.display import display
import ipywidgets as widgets
import time
import threading
import json

from pathlib import Path
from dotenv import load_dotenv

### OpenAI API Key Setup
- Securely manages OpenAI API keys by first checking for a `.env` file or environment variables. 
- If no key is found, prompts user to input their API key and automatically creates a `.env` file. 
- This ensures the API key is stored securely and persists between sessions.

In [11]:
def get_openai_api_key() -> str:
    env_path = Path('.env')
    if env_path.exists():
        load_dotenv(env_path)
    
    api_key = os.getenv('OPENAI_API_KEY')
    
    if not api_key:
        print("OpenAI API key not found in .env or environment variables.")
        print("Please enter your OpenAI API key (get one from https://platform.openai.com/api-keys):")
        api_key = input()
        
        # Write the API key to .env file
        with open(env_path, 'w') as f:
            f.write(f'OPENAI_API_KEY={api_key}')
        print(f".env file created at {env_path.absolute()}")
    
    # Debug: Show first 4 and last 4 chars of API key
    # if api_key:
    #     print(f"API Key found: {api_key[:4]}...{api_key[-4:]}")
    
    return api_key

api_key = get_openai_api_key()
api = OpenAI(api_key=api_key)

# Swarm Setup

In [7]:
client = Swarm(api)

In [8]:
print(client)
print(api)

# Triage Agent

# History Taking Agent

In [5]:
def handoff_to_medical_history_maker():
    """Hand off the OLDCART history from the history taking agent to the medical history maker."""
    return medical_history_maker_agent

In [6]:
history_taking_agent = Agent(
    name="History Taking Agent",
    instructions=("""
    Relevance:
    You are an expert medical assistant programmed to conduct detailed history-taking from patients presenting with medical complaints.
    Your goal is to gather comprehensive and precise information to assist in clinical assessment.

    Information:
    Ask targeted, sequential questions to understand the patient's symptoms. Use the OLDCART framework to structure your output:

    O: Onset
    L: Location
    D: Duration
    C: Character
    A: Aggravating factors
    R: Relieving factors
    T: Timing
    Ensure all aspects are covered before concluding the history-taking.

    Context:
    You are interacting with a patient who presents with a symptom or complaint.
    Based on their initial statement, refine your questions to explore each element of OLDCART systematically.

    Constraints:
    Be concise and avoid medical jargon that the patient may not understand.
    Ensure all questions are respectful and maintain a conversational tone.
    If the patient provides incomplete answers, prompt them with follow-up questions to clarify.

    Examples:
    Patient complaint: "I have a burning sensation when I urinate."

    Output in OLDCART format:

    Onset: When did the burning sensation begin? (e.g., "It started yesterday.")
    Location: Where exactly do you feel the burning sensation? (e.g., "In the urinary tract.")
    Duration: How long does the burning sensation last? (e.g., "Only while urinating.")
    Character: How would you describe the sensation? (e.g., "It feels like burning pain.")
    Aggravating factors: Does anything make it worse? (e.g., "Nothing specific.")
    Relieving factors: Does anything make it better? (e.g., "Nothing helps.")
    Timing: Does the sensation occur all the time or intermittently? (e.g., "Intermittently.")
    Task: Begin by asking an open-ended question like, "What seems to be the problem today?" Then, guide the conversation to gather the necessary information to fill out the OLDCART framework.
    Return a patient's history
    """
    ),
    model = "gpt-4o-mini",
    functions=[handoff_to_medical_history_maker],
)

In [27]:

def gather_history_with_OLDCART(agent, client):
    # Extended OLDCART with severity and temporality
    olcart_fields = {
        "onset": "When did you first notice these symptoms starting?",
        "location": "Where exactly do you feel these symptoms?",
        "duration": "How long have these symptoms been occurring?",
        "character": "How would you describe these symptoms (e.g., sharp, dull, burning)?",
        "aggravating factors": "What seems to make these symptoms worse?",
        "relieving factors": "Have you found anything that helps relieve these symptoms?",
        "timing": "Do these symptoms occur at a specific time or are they constant?",
        "severity": "On a scale of 0 to 10, how severe are these symptoms?",
        "temporality": "Would you describe these symptoms as constant or intermittent?"
    }

    # Track completion status for each OLDCART field
    olcart_fields_status = {field: False for field in olcart_fields.keys()}
    # Store user responses in a dictionary for final formatting
    olcart_responses = {field: None for field in olcart_fields.keys()}

    def update_olcart_status(content):
        # Simple logic: assign the current user input to the first unanswered field
        for field in olcart_fields_status:
            if not olcart_fields_status[field]:
                olcart_fields_status[field] = True
                olcart_responses[field] = content
                break

    # Initialize conversation structure
    struct = []
    initial_question = "How are you feeling today? Can you describe any symptoms or sickness you are experiencing?"
    print(initial_question)
    initial_response = input("Your response: ")
    struct.append({"role": "user", "content": initial_response})

    # Ask about each OLDCART field exactly once
    for field, question in olcart_fields.items():
        if not olcart_fields_status[field]:
            # Prompt the agent for a question about this field
            agent_prompt = (
                f"The patient has described their symptoms, but we need clarification about '{field}'. "
                "Ask one concise question to gather this detail without repeating other fields."
            )
            struct.append({"role": "system", "content": agent_prompt})
            response = client.run(agent=agent, messages=struct)
            agent_response = response.messages[-1]["content"]

            # Remove the system message to maintain a clean conversation flow
            struct.pop()

            print(agent_response)
            user_input = input("Your response: ")
            struct.append({"role": "user", "content": user_input})

            # Update OLDCART fields with the user's answer
            update_olcart_status(user_input)

    # Once done, print and also store the OLDCART history
    print("OLDCART history completed!\n")
    print("Patient History (OLDCART + Severity and Temporality):")
    print(f"Onset: {olcart_responses['onset']}")
    print(f"Location: {olcart_responses['location']}")
    print(f"Duration: {olcart_responses['duration']}")
    print(f"Character: {olcart_responses['character']}")
    print(f"Aggravating factors: {olcart_responses['aggravating factors']}")
    print(f"Relieving factors: {olcart_responses['relieving factors']}")
    print(f"Severity: {olcart_responses['severity']}")
    print(f"Temporality <constant | intermittent>: {olcart_responses['temporality']}")
    print(f"Timing: {olcart_responses['timing']}\n")

    # Add a final assistant message with structured OLDCART data
    final_olcart_data = {
        "onset": olcart_responses["onset"],
        "location": olcart_responses["location"],
        "duration": olcart_responses["duration"],
        "character": olcart_responses["character"],
        "aggravating factors": olcart_responses["aggravating factors"],
        "relieving factors": olcart_responses["relieving factors"],
        "severity": olcart_responses["severity"],
        "temporality": olcart_responses["temporality"],
        "timing": olcart_responses["timing"]
    }

    structured_message = {
        "role": "assistant",
        "content": json.dumps(final_olcart_data, indent=2)
    }
    struct.append(structured_message)

    print("Conversation history:")
    for message in struct:
        print(f"{message['role'].capitalize()}: {message['content']}")

    return struct

In [28]:
# Example usage
gathered_data = gather_history_with_OLDCART(agent=history_taking_agent, client=client)
print(gathered_data)

How are you feeling today? Can you describe any symptoms or sickness you are experiencing?
Your response: i cant hear on my left ear
When did you first notice the hearing loss in your left ear?
Your response: 2 days ago
Where exactly do you feel the issue in your left ear?
Your response: inside my ear
How long have you been experiencing this hearing loss in your left ear?
Your response: 2 days
How would you describe the hearing loss in your left ear? For example, does it feel muffled or like a complete loss of sound?
Your response: muffled 
What makes the muffled hearing in your left ear worse?
Your response: when i blow my nose or being near my cat
What, if anything, makes the muffled feeling in your ear better?
Your response: yawning
Does the muffled feeling occur all the time, or does it come and go?
Your response: all the time
On a scale of 1 to 10, how severe would you say the muffled sensation in your ear is, with 10 being the worst?
Your response: 8
Can you describe if the muffl

In [31]:
gathered_data

[{'role': 'user', 'content': 'i cant hear on my left ear'},
 {'role': 'user', 'content': '2 days ago'},
 {'role': 'user', 'content': 'inside my ear'},
 {'role': 'user', 'content': '2 days'},
 {'role': 'user', 'content': 'muffled '},
 {'role': 'user', 'content': 'when i blow my nose or being near my cat'},
 {'role': 'user', 'content': 'yawning'},
 {'role': 'user', 'content': 'all the time'},
 {'role': 'user', 'content': '8'},
 {'role': 'user', 'content': 'constant'},
 {'role': 'assistant',
  'content': '{\n  "onset": "2 days ago",\n  "location": "inside my ear",\n  "duration": "2 days",\n  "character": "muffled ",\n  "aggravating factors": "when i blow my nose or being near my cat",\n  "relieving factors": "yawning",\n  "severity": "8",\n  "temporality": "constant",\n  "timing": "all the time"\n}'}]

# Medical History Maker Agent

In [19]:
def handoff_to_assessment_agent():
    """Hand off the Patient Medical history to the Assessment Agent who will identify differentials diagnosis."""
    return assessment_agent

In [20]:
medical_history_maker_agent = Agent(
    name="Medical History Making Agent",
    instructions=("""
R (Role):
You are an expert emergency room doctor, a specialized medical assistant designed to collect and synthesize a patient’s symptom history using the OLDCARTS framework. Your purpose is to engage with a patient or provided data and produce a structured, medically coherent narrative that a healthcare professional can quickly review and understand.

I (Input):
You will receive some initial patient input describing symptoms and possibly partial OLDCARTS details. The patient may have already described one or more components of OLDCARTS (Onset, Location, Duration, Character, Aggravating factors, Relieving factors, Timing, Severity, Temporality). After you gather this initial data, you should identify what is missing and only ask for the outstanding details. If certain critical points remain ambiguous or incomplete after the patient’s responses, you may ask up to three brief clarifying questions. Once all necessary information is collected, you will finalize the patient’s history, incorporating all OLDCARTS elements into a single coherent narrative.

C (Context):
You are operating in a medical context, providing a tool for clinicians. The final output is intended for a clinical audience—medical professionals who want a clear, concise, and logically organized summary of the patient’s presenting problem. The patient’s details may vary: they could be vague or specific, and they may or may not spontaneously provide all necessary OLDCARTS information. You have the ability to ask questions, but you must be mindful to ask only the minimal number of questions needed.

C (Constraints):
Completeness of OLDCARTS: Ensure that you have collected Onset, Location, Duration, Character, Aggravating factors, Relieving factors, Timing, Severity, and Temporality.
Minimal Questioning: Only ask about fields that have not been clearly answered. Avoid re-asking for details already provided by the patient.
Limited Clarification: If critical details are unclear after initial questioning, ask no more than three additional clarifying questions. If still unclear, finalize the history with the best available data.
Professional Tone: The final narrative should be neutral, professional, and medically appropriate. Use standard clinical language understandable to healthcare providers.
Concise but Complete: The final output should be a well-structured paragraph (or short set of paragraphs) summarizing the chief complaint and the details collected through the OLDCARTS framework. Do not merely list the data points; integrate them into a coherent, narrative-style History of Present Illness (HPI).
No Speculation: Base the summary only on the information provided by the patient. Do not add unverified assumptions.

E (Evaluation):
Your output is successful if it:
Reflects a clearly identified chief complaint.
Incorporates all OLDCARTS elements, provided or elicited, in a coherent narrative.
Requires no further clarification from a clinical perspective, or if still ambiguous, has made a good faith effort to clarify and then presented the best possible summary.
Is well-organized, logically consistent, and easy for a clinician to read quickly.
Uses a factual, neutral tone without extraneous details or speculation.

Sample output could look something like the following:

Patient History:
Chief Complaint: The patient, a 35-year-old female, presents with a severe burning pain during urination.
History of Present Illness: The patient reports that the symptoms began less than one day ago. She describes the pain as a burning sensation localized to the urinary tract, occurring only during urination. The pain is intermittent, with episodes of intense discomfort rated 9 out of 10 in severity. There are no known aggravating or relieving factors.
Additional Information:
Past Medical History: The patient has no significant past medical history.
Medications: She is not currently taking any medications.
Allergies: The patient has no known allergies.
Social History: She is a non-smoker and consumes alcohol occasionally.
Family History: There is no family history of urinary tract infections.

    """
    ),
    model = "gpt-4o-mini",
    functions=[handoff_to_assessment_agent],
)

In [32]:
response = client.run(agent=medical_history_maker_agent, messages=gathered_data)
print(response.messages[0]["content"])

**Patient History:**

**Chief Complaint:** The patient, a currently unspecified individual, presents with a complaint of hearing loss in the left ear.

**History of Present Illness:** The patient reports that the symptoms began 2 days ago, describing the sensation as a muffled sound localized to the inside of the left ear. The hearing loss has persisted constantly over this duration, with a severity rating of 8 out of 10. The patient notes that the symptoms are aggravated by actions such as blowing the nose or being near their cat, while yawning provides some relief. 

No additional information regarding past medical history, medications, allergies, social or family history has been provided. 

Would you like me to proceed with further assessment?


TODO: fix hallucination if we need more info from patient

# Assessment Agent

In [23]:
def handoff_to_treatment_agent():
    """Hand off the Treatment Agent who will suggest treatment based on the diagnosis of the patient."""
    return treatment_agent

In [24]:
assessment_agent = Agent(
    name="Assessment Agent",
    instructions=("""
R (Role):
You are the Assessment Agent, a medical reasoning assistant trained to analyze patient histories and suggest potential diagnoses or differential diagnoses. Your purpose is to synthesize provided clinical information into a medically sound, prioritized list of possible conditions based on likelihood and relevance.

I (Input):
You will receive a detailed patient history, including symptoms, onset, location, duration, character, aggravating factors, relieving factors, timing, severity, temporality (OLDCARTS), as well as any additional pertinent medical details (past medical history, medications, allergies, social history, family history).
Your input may also include the patient’s demographic information such as age, sex, and relevant lifestyle factors.

C (Context):
The Assessment Agent operates within a clinical reasoning framework. Your output is intended for healthcare professionals who will use this information to guide further diagnostic workup. The patient history provided should be considered as raw input data. You should generate a list of likely differential diagnoses that best fit the provided clinical picture. If uncertain, you may highlight areas where more information would be helpful, but still provide a reasoned differential based on the available data.

C (Constraints):

Clinical Accuracy: Ground your reasoning in standard medical knowledge and common clinical reasoning patterns. Avoid purely speculative diagnoses unsupported by the details provided.
Prioritization: Rank or highlight the most likely diagnoses first, explaining briefly why they are plausible given the patient’s presentation. Less likely possibilities can be mentioned subsequently.
Clarity & Brevity: Present the differentials in a concise, organized manner. Include a short rationale for each primary option.
Non-Definitive: Do not provide a definitive diagnosis. Instead, present a range of reasonable considerations. It’s acceptable to mention that further tests or information would be needed to refine the assessment.
No Patient Management Instructions: Focus on the diagnostic reasoning aspect. Do not provide treatment plans or management suggestions. Only discuss likely or possible diagnoses.
E (Evaluation):
Your output is successful if it:

Reflects a sound clinical reasoning process.
Lists likely differential diagnoses in an organized and prioritized fashion.
Uses medically appropriate terminology and is understandable by healthcare professionals.
Remains within the information provided and avoids unfounded speculation.
Is concise, coherent, and sufficiently explanatory to justify the presence of each suggested diagnosis.
    """
    ),
    model = "gpt-4o-mini",
    functions=[handoff_to_assessment_agent],
)

In [33]:
# usage

# Step 1: Gather OLDCART History from User via History Taking Agent
# gathered_data = gather_history_with_OLDCART(agent=history_taking_agent, client=client)

# The `gathered_data` structure now contains a series of messages between user and the agent.
# The last message should be the assistant message containing the OLDCART data in JSON.
final_olcart_data_str = None
for msg in reversed(gathered_data):
    if msg["role"] == "assistant":
        # Attempt to parse the content as JSON
        try:
            olcart_data = json.loads(msg["content"])
            final_olcart_data_str = msg["content"]  # Keep the raw JSON string to pass along
            break
        except json.JSONDecodeError:
            # Not the structured OLDCART data, move on
            continue

if not final_olcart_data_str:
    raise ValueError("No structured OLDCART data found in the conversation.")

# Step 2: Assess the Patient's History Using the Assessment Agent
# Now we take the structured OLDCART data and pass it to the assessment agent
assessment_messages = [
    {
        "role": "system",
        "content": (
            "You are the Assessment Agent. Given the patient's OLDCARTS history data, "
            "propose a list of possible differential diagnoses, in order of likelihood, "
            "with brief rationales for each. Do not provide a definitive diagnosis. "
            "If information is missing or ambiguous, note it, but do your best with the given data."
        )
    },
    {
        "role": "user",
        "content": final_olcart_data_str  # The JSON string from the history-taking step
    }
]

assessment_response = client.run(agent=assessment_agent, messages=assessment_messages)
assessment_output = assessment_response.messages[-1]["content"]
print("Assessment Agent Output:\n", assessment_output)

# The assessment_output should contain a list of differential diagnoses in some textual format.
# For better downstream usage, you might want the assessment agent to return a structured format (JSON),
# but here we assume it returns a readable text format.

# Step 3: Use the Treatment Agent to Suggest Next Steps Based on Differentials
# treatment_messages = [
#     {
#         "role": "system",
#         "content": (
#             "You are the Treatment Agent. Given the differential diagnoses provided, "
#             "suggest appropriate next steps, which could include further diagnostic testing, "
#             "initial management, or specialist referrals. Do not finalize a single diagnosis, "
#             "but guide the next reasonable clinical steps."
#         )
#     },
#     {
#         "role": "user",
#         "content": assessment_output  # The text output from the assessment agent
#     }
# ]

# treatment_response = client.run(agent=treatment_agent, messages=treatment_messages)
# treatment_output = treatment_response.messages[-1]["content"]
# print("Treatment Agent Output:\n", treatment_output)

# At this point, you have:
# - A structured OLDCART history from the first agent
# - A set of possible differentials from the second agent
# - Suggested next steps from the treatment agent

# This completes the pipeline of connecting the three agents together.


Assessment Agent Output:
 Based on the provided OLDCARTS history, here are the likely differential diagnoses:

1. **Eustachian Tube Dysfunction**:
   - **Rationale**: The patient reports muffled hearing that worsens with actions like blowing the nose, suggesting an obstruction or dysfunction in the eustachian tube. Yawning relieving the symptoms also supports this diagnosis, as yawning can help equalize pressure in the ear.

2. **Otitis Media**:
   - **Rationale**: Given the severity (8/10) and nature (muffled) of the symptoms, a middle ear infection may be present. This condition often follows upper respiratory tract infections, leading to fluid accumulation and muffled hearing.

3. **Allergic Rhinitis or Otitis Media with Effusion**:
   - **Rationale**: The aggravation of symptoms when near the cat suggests a possible allergic component. Allergies can lead to fluid buildup in the middle ear (otitis media with effusion), resulting in muffled hearing.

4. **Cerumen Impaction**:
   - **

In [34]:
def run_assessment_phase(gathered_data, assessment_agent, client):
    # Extract the final OLDCART JSON data from the gathered_data
    final_olcart_data_str = None
    for msg in reversed(gathered_data):
        if msg["role"] == "assistant":
            try:
                # Attempt to parse the assistant message content as JSON
                json.loads(msg["content"])
                final_olcart_data_str = msg["content"]
                break
            except json.JSONDecodeError:
                # Not structured OLDCART data, continue searching
                continue

    if not final_olcart_data_str:
        raise ValueError("No structured OLDCART data found in the conversation.")

    # Define the messages for the assessment agent
    assessment_messages = [
        {
            "role": "system",
            "content": (
                "You are the Doctor, Medical Assessment Agent. Given the patient's OLDCARTS history data, "
                "propose a list of possible differential diagnoses, in order of likelihood, "
                "with brief rationales for each. Do not provide a definitive diagnosis. "
                "If information is missing or ambiguous, note it, but do your best with the given data."
            )
        },
        {
            "role": "user",
            "content": final_olcart_data_str
        }
    ]

    # Run the assessment agent
    assessment_response = client.run(agent=assessment_agent, messages=assessment_messages)
    assessment_output = assessment_response.messages[-1]["content"]
    print("Assessment Agent Output:\n", assessment_output)

    return assessment_output

In [35]:
# Example usage:
# gathered_data = gather_history_with_OLDCART(agent=history_taking_agent, client=client)
assessment_output = run_assessment_phase(gathered_data, assessment_agent, client)
# assessment_output now contains the differential diagnoses from the Assessment Agent


Assessment Agent Output:
 Based on the provided OLDCARTS history, here is a prioritized list of possible differential diagnoses with brief rationales:

1. **Eustachian Tube Dysfunction**:
   - Rationale: The symptoms of muffled hearing and aggravation with nose blowing suggest that the Eustachian tube may be blocked or not functioning properly, which can cause pressure changes and muffled hearing. The constant nature and severity (8 out of 10) align with this condition.

2. **Earwax Impaction (Cerumen Impaction)**:
   - Rationale: The description of muffled hearing could be consistent with earwax blocking the ear canal. This could also explain the discomfort when near an allergen (the cat), as allergens can influence earwax production and sensation.

3. **Middle Ear Infection (Otitis Media)**:
   - Rationale: This could cause similar symptoms, such as muffled hearing and pain/discomfort, although the absence of pain or fever makes it less likely. The duration of symptoms supports the c

# Treatment Agent

In [36]:
def handoff_to_medication_agent():
    """Hand off the Medication List Agent, a RAG model who will suggest what meds to take based on the assessment of the patient."""
    return medication_agent

In [37]:
treatment_agent = Agent(
    name="Treatment Agent",
    instructions=("""
R (Role):
You are the Treatment Agent, acting in the role of a medical doctor. Your task is to recommend appropriate next steps in patient management based on the differential diagnoses provided.

I (Input):
You will be given the assessment results from the Assessment Agent, which detail a list of possible differential diagnoses and their rationales. You must use this information to guide initial management and clinical decision-making.

C (Context):
The context is a clinical environment where a patient has presented with certain symptoms and an Assessment Agent has provided a set of likely differential diagnoses. Your job is not to definitively diagnose the patient but to offer reasonable, evidence-based next steps that a physician might take to clarify, evaluate, or manage the patient’s condition. These steps might include ordering specific tests, recommending conservative management measures, or suggesting referrals to specialists if appropriate.

C (Constraints):

Evidence-Based and Safe: Provide standard-of-care, reasonable steps without overreaching.
No Definitive Diagnosis: Do not finalize a single, definitive diagnosis; maintain the differential nature.
Focus on Next Steps: Suggest further diagnostic tests, physical exams, initial treatments, lifestyle modifications, or referrals as would be appropriate in a clinical setting.
Acknowledge Uncertainty: If certain information is missing, note what would be helpful to obtain next.
Concise and Professional: Keep the guidance straightforward, actionable, and medically appropriate.
E (Evaluation):
Your output is successful if it:

Reflects a clear, clinically logical set of next steps.
Aligns with standard medical reasoning and patient care pathways.
Avoids speculation beyond the provided differentials.
Offers a concise, clinically relevant plan that another medical professional would find helpful.

    """
    ),
    model = "gpt-4o-mini",
    functions=[handoff_to_medication_agent],
)

In [38]:
# Step 3: Use the Treatment Agent with the Assessment Output
treatment_messages = [
    {
        "role": "system",
        "content": (
            "You are the Treatment Agent, acting as a medical doctor. "
            "Given the patient's differential diagnoses from the Assessment Agent, "
            "propose initial next steps or management strategies. "
            "These could include further testing, conservative treatments, referrals, "
            "or suggestions for additional information needed."
        )
    },
    {
        "role": "user",
        "content": assessment_output
    }
]

treatment_response = client.run(agent=treatment_agent, messages=treatment_messages)
treatment_output = treatment_response.messages[-1]["content"]
print("Treatment Agent Output:\n", treatment_output)

Treatment Agent Output:
 Based on the differential diagnoses provided, here are the recommended next steps for patient management:

1. **Ear Examination**:
   - Perform an otoscopic examination to check for signs of earwax impaction, middle ear infection (e.g., tympanic membrane redness, bulging), or any other abnormalities in the ear canal or ear drum.

2. **Audiometric Testing**:
   - If initial examination does not reveal obvious causes, consider arranging for an audiogram to evaluate the extent of hearing loss and determine if it corresponds with Eustachian tube dysfunction or other auditory conditions.

3. **Nasal Examination**:
   - Assess for nasal congestion or discharge that could indicate allergic rhinitis or acute sinusitis. This can include a physical examination of the nasal passages.

4. **Allergy Testing**:
   - If allergic rhinitis is suspected, consider skin prick testing or specific IgE testing for common allergens, including cat dander, to confirm the allergy.

5. **

# Medication List Agent + RAG Model

# Basic Prescription Writer Agent